In [1]:
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import os
import pandas as pd  
import tensorflow_addons as tfa
import numpy as np
import matplotlib.pyplot as plt
from tensorflow.keras import layers, models  # Ensure this line is included
from sklearn.metrics import precision_score, recall_score, f1_score, accuracy_score, confusion_matrix, classification_report



2024-11-16 23:41:05.643536: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-11-16 23:41:06.051489: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2024-11-16 23:41:06.051505: I tensorflow/compiler/xla/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.
2024-11-16 23:41:07.436952: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: libnvinfer.so.7: cannot open shared object file: No such file or directory
2024-

In [2]:
base_dir = r'/home/prashantb/Documents/Prashant/Thesis/FinalData'

train_dir = os.path.join(base_dir, 'train')
test_dir = os.path.join(base_dir, 'test')

In [3]:
# Data Augmentation for training
train_datagen = ImageDataGenerator(
    rescale=1./255,
    rotation_range=40,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    fill_mode='nearest',
    validation_split=0.2  # 20% of training data used as validation data
)

In [4]:
# Data generator for training data
train_generator = train_datagen.flow_from_directory(
    '/home/prashantb/Documents/Prashant/Thesis/FinalData/train',
    target_size=(128, 128),
    batch_size=32,
    class_mode='binary',
    subset='training'  # Set as training data
)

Found 13964 images belonging to 2 classes.


In [5]:
# Data generator for validation data
validation_generator = train_datagen.flow_from_directory(
    '/home/prashantb/Documents/Prashant/Thesis/FinalData/train',
    target_size=(128, 128),
    batch_size=16,
    class_mode='binary',
    subset='validation'  # Set as validation data
)

Found 3490 images belonging to 2 classes.


In [6]:
# Data generator for test data (no augmentation)
test_datagen = ImageDataGenerator(rescale=1./255)

test_generator = test_datagen.flow_from_directory(
    '/home/prashantb/Documents/Prashant/Thesis/FinalData/test',
    target_size=(128, 128),
    batch_size=16,
    class_mode='binary'
)

Found 2096 images belonging to 2 classes.


In [ ]:
import tensorflow as tf
from tensorflow.keras import layers, models
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, GlobalAveragePooling2D, Dense, Dropout, BatchNormalization, Input, Concatenate
from tensorflow.keras.preprocessing.image import ImageDataGenerator

# CNN Model Definition
cnn_model = Sequential([
    Conv2D(64, (3, 3), activation='relu', input_shape=(128, 128, 3), kernel_regularizer=tf.keras.regularizers.l2(0.001)),
    BatchNormalization(),
    Conv2D(64, (3, 3), activation='relu', kernel_regularizer=tf.keras.regularizers.l2(0.001)),
    BatchNormalization(),
    MaxPooling2D((2, 2)),
    Dropout(0.1),

    Conv2D(128, (3, 3), activation='relu', kernel_regularizer=tf.keras.regularizers.l2(0.001)),
    BatchNormalization(),
    Conv2D(128, (3, 3), activation='relu', kernel_regularizer=tf.keras.regularizers.l2(0.001)),
    BatchNormalization(),
    MaxPooling2D((2, 2)),
    Dropout(0.1),

    Conv2D(256, (3, 3), activation='relu', kernel_regularizer=tf.keras.regularizers.l2(0.001)),
    BatchNormalization(),
    Conv2D(256, (3, 3), activation='relu', kernel_regularizer=tf.keras.regularizers.l2(0.001)),
    BatchNormalization(),
    MaxPooling2D((2, 2)),
    Dropout(0.1),

    Conv2D(512, (3, 3), activation='relu', kernel_regularizer=tf.keras.regularizers.l2(0.001)),
    BatchNormalization(),
    Conv2D(512, (3, 3), activation='relu', kernel_regularizer=tf.keras.regularizers.l2(0.001)),
    BatchNormalization(),
    MaxPooling2D((2, 2)),
    Dropout(0.1),

    GlobalAveragePooling2D(),

    Dense(2048, activation='relu'),
    Dropout(0.5),

    Dense(1, activation='sigmoid')
])

# Vision Transformer (Hybrid) Model Definition
num_heads = 8
num_layers = 6
mlp_dim = 2048
hidden_dim = 512
patch_size = 16
num_patches = (128 // patch_size) ** 2  # (128 / 16)^2 = 64 patches
dropout_rate = 0.1

# Patch and Position Embedding Layer
class PatchEmbedding(layers.Layer):
    def __init__(self, num_patches, projection_dim):
        super(PatchEmbedding, self).__init__()
        self.num_patches = num_patches
        self.projection = layers.Dense(units=projection_dim)
        self.position_embedding = layers.Embedding(
            input_dim=num_patches, output_dim=projection_dim
        )

    def call(self, patch):
        positions = tf.range(start=0, limit=self.num_patches, delta=1)
        encoded = self.projection(patch) + self.position_embedding(positions)
        return encoded

# Transformer Block
def transformer_block(inputs, num_heads, mlp_dim, dropout_rate):
    x1 = layers.LayerNormalization(epsilon=1e-6)(inputs)
    attention_output = layers.MultiHeadAttention(
        num_heads=num_heads, key_dim=hidden_dim, dropout=dropout_rate
    )(x1, x1)
    x2 = layers.Add()([attention_output, inputs])

    x3 = layers.LayerNormalization(epsilon=1e-6)(x2)
    x3 = layers.Dense(mlp_dim, activation=tf.nn.gelu)(x3)
    x3 = layers.Dropout(dropout_rate)(x3)
    x3 = layers.Dense(hidden_dim)(x3)
    return layers.Add()([x3, x2])

# ViT Model
def create_vit_classifier():
    inputs = layers.Input(shape=(128, 128, 3))
    patches = layers.Conv2D(hidden_dim, kernel_size=patch_size, strides=patch_size)(inputs)
    patches = layers.Reshape((num_patches, hidden_dim))(patches)
    encoded_patches = PatchEmbedding(num_patches, hidden_dim)(patches)

    for _ in range(num_layers):
        encoded_patches = transformer_block(encoded_patches, num_heads, mlp_dim, dropout_rate)

    representation = layers.LayerNormalization(epsilon=1e-6)(encoded_patches)
    representation = layers.Flatten()(representation)
    representation = layers.Dropout(0.5)(representation)
    features = layers.Dense(mlp_dim, activation=tf.nn.gelu)(representation)
    logits = layers.Dense(1)(features)
    outputs = layers.Activation("sigmoid")(logits)
    return models.Model(inputs=inputs, outputs=outputs)

# Create ViT model
vit_model = create_vit_classifier()

# Hybrid Model
input_layer = Input(shape=(128, 128, 3))

# CNN Model Output
cnn_output = cnn_model(input_layer)

# ViT Model Output
vit_output = vit_model(input_layer)

# Combine outputs from CNN and ViT
combined_features = Concatenate()([cnn_output, vit_output])

# Additional Dense Layers for the final classification
x = Dense(1024, activation='relu')(combined_features)
x = Dropout(0.5)(x)
x = Dense(512, activation='relu')(x)
x = Dropout(0.5)(x)

# Final output layer for binary classification
output_layer = Dense(1, activation='sigmoid')(x)

# Create and compile the hybrid model
hybrid_model = models.Model(inputs=input_layer, outputs=output_layer)

# Compile the hybrid model
hybrid_model.compile(
    optimizer=tf.keras.optimizers.Adam(learning_rate=1e-4),
    loss='binary_crossentropy',
    metrics=['accuracy']
)

# Print the model summary
hybrid_model.summary()

# Data Generators for Training, Validation, and Testing
train_datagen = ImageDataGenerator(
    rescale=1./255,
    rotation_range=40,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    fill_mode='nearest',
    validation_split=0.2
)

train_generator = train_datagen.flow_from_directory(
    '/home/prashantb/Documents/Prashant/Thesis/FinalData/train',
    target_size=(128, 128),
    batch_size=32,
    class_mode='binary',
    subset='training'
)

validation_generator = train_datagen.flow_from_directory(
    '/home/prashantb/Documents/Prashant/Thesis/FinalData/train',
    target_size=(128, 128),
    batch_size=16,
    class_mode='binary',
    subset='validation'
)

test_datagen = ImageDataGenerator(rescale=1./255)

test_generator = test_datagen.flow_from_directory(
    '/home/prashantb/Documents/Prashant/Thesis/FinalData/test',
    target_size=(128, 128),
    batch_size=16,
    class_mode='binary'
)



2024-11-16 23:41:39.882048: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcuda.so.1'; dlerror: libcuda.so.1: cannot open shared object file: No such file or directory
2024-11-16 23:41:39.882345: W tensorflow/compiler/xla/stream_executor/cuda/cuda_driver.cc:265] failed call to cuInit: UNKNOWN ERROR (303)
2024-11-16 23:41:39.882360: I tensorflow/compiler/xla/stream_executor/cuda/cuda_diagnostics.cc:156] kernel driver does not appear to be running on this host (prashantB-viveka): /proc/driver/nvidia/version does not exist
2024-11-16 23:41:39.883297: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


Model: "model_1"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_2 (InputLayer)           [(None, 128, 128, 3  0           []                               
                                )]                                                                
                                                                                                  
 sequential (Sequential)        (None, 1)            5745729     ['input_2[0][0]']                
                                                                                                  
 model (Functional)             (None, 1)            130822145   ['input_2[0][0]']                
                                                                                                  
 concatenate (Concatenate)      (None, 2)            0           ['sequential[0][0]',       

In [8]:
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint

early_stopping = EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True)
model_checkpoint = ModelCheckpoint('best_model.h5', save_best_only=True, monitor='val_loss')

# Train the hybrid model
history = hybrid_model.fit(
    train_generator,
    epochs=25,  # Adjust epochs as needed
    validation_data=validation_generator
)

# Log the performance
import logging

logging.basicConfig(filename='hybrid_model_performance.log', level=logging.INFO)

logging.info(f"Training Accuracy: {history.history['accuracy']}")
logging.info(f"Validation Accuracy: {history.history['val_accuracy']}")
logging.info(f"Training Loss: {history.history['loss']}")
logging.info(f"Validation Loss: {history.history['val_loss']}")


Epoch 1/25
437/437 [==============================] - 3191s 7s/step - loss: 1.3604 - accuracy: 0.7119 - val_loss: 1.2434 - val_accuracy: 0.7146
Epoch 2/25
437/437 [==============================] - 3115s 7s/step - loss: 0.9151 - accuracy: 0.7375 - val_loss: 0.8477 - val_accuracy: 0.7590
Epoch 3/25
437/437 [==============================] - 3114s 7s/step - loss: 0.7642 - accuracy: 0.7767 - val_loss: 0.6454 - val_accuracy: 0.8458
Epoch 4/25
437/437 [==============================] - 3115s 7s/step - loss: 0.6838 - accuracy: 0.7928 - val_loss: 0.5212 - val_accuracy: 0.8894
Epoch 5/25
437/437 [==============================] - 3117s 7s/step - loss: 0.6402 - accuracy: 0.7965 - val_loss: 0.5239 - val_accuracy: 0.8880
Epoch 6/25
437/437 [==============================] - 3117s 7s/step - loss: 0.6071 - accuracy: 0.8025 - val_loss: 0.4307 - val_accuracy: 0.8968
Epoch 7/25
437/437 [==============================] - 3116s 7s/step - loss: 0.5851 - accuracy: 0.8054 - val_loss: 0.4996 - val_accuracy:

In [9]:
from sklearn.metrics import classification_report, confusion_matrix

y_true = test_generator.classes
y_pred = hybrid_model.predict(test_generator)
y_pred = (y_pred > 0.5).astype(int)  # Convert probability to binary class

print("Classification Report:")
print(classification_report(y_true, y_pred))
print("Confusion Matrix:")
print(confusion_matrix(y_true, y_pred))

# Evaluate the model on the validation data
y_true = validation_generator.classes
y_pred = (hybrid_model.predict(validation_generator) > 0.5).astype("int32")

# Compute metrics
accuracy = accuracy_score(y_true, y_pred)
precision = precision_score(y_true, y_pred)
recall = recall_score(y_true, y_pred)
f1 = f1_score(y_true, y_pred)

# Log metrics
logging.info(f'Hybrid Model - Accuracy: {accuracy:.4f}')
logging.info(f'Hybrid Model - Precision: {precision:.4f}')
logging.info(f'Hybrid Model - Recall: {recall:.4f}')
logging.info(f'Hybrid Model - F1-score: {f1:.4f}')

# Output results to the console
print(f"Hybrid Model - Accuracy: {accuracy:.4f}")
print(f"Hybrid Model - Precision: {precision:.4f}")
print(f"Hybrid Model - Recall: {recall:.4f}")
print(f"Hybrid Model - F1-score: {f1:.4f}")


y_pred = (vit_model.predict(validation_generator) > 0.5).astype("int32")

# Compute metrics
accuracy = accuracy_score(y_true, y_pred)
precision = precision_score(y_true, y_pred)
recall = recall_score(y_true, y_pred)
f1 = f1_score(y_true, y_pred)

# Log metrics
logging.info(f'ViT Model - Accuracy: {accuracy:.4f}')
logging.info(f'ViT Model - Precision: {precision:.4f}')
logging.info(f'ViT Model - Recall: {recall:.4f}')
logging.info(f'ViT Model - F1-score: {f1:.4f}')

# Output results to the console
print(f"ViT Model - Accuracy: {accuracy:.4f}")
print(f"ViT Model - Precision: {precision:.4f}")
print(f"ViT Model - Recall: {recall:.4f}")
print(f"ViT Model - F1-score: {f1:.4f}")



131/131 [==============================] - 131s 958ms/step
Classification Report:
              precision    recall  f1-score   support

           0       0.32      0.33      0.33       701
           1       0.66      0.66      0.66      1395

    accuracy                           0.55      2096
   macro avg       0.49      0.49      0.49      2096
weighted avg       0.55      0.55      0.55      2096

Confusion Matrix:
[[229 472]
 [479 916]]
219/219 [==============================] - 230s 1s/step
Hybrid Model - Accuracy: 0.5971
Hybrid Model - Precision: 0.7130
Hybrid Model - Recall: 0.7302
Hybrid Model - F1-score: 0.7215
219/219 [==============================] - 173s 783ms/step
ViT Model - Accuracy: 0.2854
ViT Model - Precision: 0.0000
ViT Model - Recall: 0.0000
ViT Model - F1-score: 0.0000


/home/prashantb/.local/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
